# Set 1

## Task 1: *convert hex to base64*

In [24]:
input_hex = "49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d"

In [26]:
import base64
print(input_bytes := base64.b16decode(input_hex, casefold=True))

b"I'm killing your brain like a poisonous mushroom"


In [31]:
output_b64 = base64.b64encode(input_bytes).decode()
correct_output = "SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t"
if correct_output == output_b64:
    print("Task 1 passed successfully!")

Task 1 passed successfully!


## Task 2: *Fixed XOR*

In [32]:
a = "1c0111001f010100061a024b53535009181c"
b = "686974207468652062756c6c277320657965"

Time to get familiar with a great quirk of Python: bit-wise operations take place on *integers*, not on *bytes* objects! So we have to take the input bytes, read them as integers and then get back to hex to check the result.

In [40]:
int_a = int(a, 16)
int_b = int(b, 16)
result = hex(int_a ^ int_b)[2:]

746865206b696420646f6e277420706c6179


Here, the slicing on the string is to remove the characters that Python naturally adds to the front to let us know this is a hex-encoded string.

In [41]:
correct_output = "746865206b696420646f6e277420706c6179"
if correct_output == result:
    print("Task 2 passed successfully!")


Task 2 passed successfully!


## Task 3: *Single-byte XOR cipher*

The first step is to work out what a good English-language plaintext should look like. We can do this by sampling from somethiing like "The Lord of the Rings" to get a Dictionary of letter frequencies:

In [7]:
english_chars = "abcdefghijklmnopqrstuvwxyz "
frequency_dict = {c:0 for c in english_chars}
with open("lotr.txt", "r") as lotr_file:
    total = 0
    for line in lotr_file:
        for c in line:
            if c.lower() in english_chars:
                frequency_dict[c.lower()] += 1
                total += 1

print(frequency_dict := {c:frequency_dict[c]/total for c in english_chars})
print(f"The total of the normalised frequencies sum to {sum(frequency_dict.values())}")

{'a': 0.0631495689263029, 'b': 0.012904205449377615, 'c': 0.012387804122339521, 'd': 0.039675810045899824, 'e': 0.09336730250401223, 'f': 0.01811969697116862, 'g': 0.018872444987070542, 'h': 0.050483652741314014, 'i': 0.046955288062905776, 'j': 0.000668245980568417, 'k': 0.007006546155756353, 'l': 0.033543422876352154, 'm': 0.017975622619737928, 'n': 0.052605268977101244, 'o': 0.05926312282613658, 'p': 0.009846591573172126, 'q': 0.0004276903780672863, 'r': 0.04429007444273174, 's': 0.04539572367899199, 't': 0.06812515237076773, 'u': 0.02009464875482528, 'v': 0.0061498703492718415, 'w': 0.020826999727715664, 'x': 0.0003713556878449488, 'y': 0.01481893739176212, 'z': 0.00032117248678482056, ' ': 0.24235377991202076}
The total of the normalised frequencies sum to 1.0


Now, we can loop over the characters in *english_chars* and do the following:
1. generate the *XOR_combo* of that character with the given ciphertext;
2. compare, using *SciPy*'s built-in *chisquare* test, the letter frequencies recorded for the 'decoded' text with the letter frequencies from "The Lord of the Rings";

and consequently pick the one with the lowest Chi-Square score to be the correct plaintext!

First, we need to get *SciPy*'s *chisquare* function:

In [8]:
from scipy.stats import chisquare

Now we want to decipher the ciphertext, given below as a hex-encoded string:

In [13]:
ciphertext = "1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736"

In [23]:
inf_ch_sq = 1000
best_decipher = None
key = None
for c in english_chars:
    total = 0
    local_dict = {a:0 for a in english_chars}
    XOR_text = functions.hex_to_string(functions.single_byte_XOR(functions.hex_to_string(ciphertext), c))
    for d in XOR_text:
        if d.lower() in english_chars:
            local_dict[d.lower()] += 1
            total += 1
    local_dict = {c:local_dict[c]/total for c in english_chars}
    ch_sq = chisquare(list(local_dict.values()), list(frequency_dict.values()))[0]
    if ch_sq < inf_ch_sq:
        inf_ch_sq = ch_sq
        best_decipher = XOR_text
        key = c

print(f"The key was '{key}', with a Chi-Square value of {ch_sq:.3f}")
print("The corresponding plaintext was:")
print(best_decipher)    

The key was 'x', with a Chi-Square value of 2691.836
The corresponding plaintext was:
cOOKING mcS LIKE A POUND OF BACON


In [11]:
import importlib
importlib.reload(functions)

<module 'functions' from 'c:\\Users\\whatf\\OneDrive\\Desktop\\Cryptopals-lite\\functions.py'>